The Sportradar Soccer v4 API was used for this notebook to obtain data from the 2020-2021 season for the English Premier League. You can obtain your own API Key at https://api.sportradar.com/soccer/trial/v4/openapi/swagger/index.html

The goal of this exercise was to create a standings table that consists of the following information:
- abbreviation of team name
- games played
- games won
- games drawn (ties)
- games lost
- goals for (total made) 
- goals against (total conceded)
- goal difference
- total points

In [1]:
soccer_key = "input your API here"

In [2]:
import requests

def get_seasons_list(soccer_key):
    address = "https://api.sportradar.com/soccer/trial/v4/en/seasons.json?api_key=" + soccer_key
    return requests.get(address).json()

all_seasons = get_seasons_list(soccer_key)
[x for x in all_seasons['seasons'] if 'premier' in x['name'].lower()][:5]

[{'id': 'sr:season:87140',
  'name': 'Premier League 21/22',
  'start_date': '2021-10-16',
  'end_date': '2022-05-31',
  'year': '21/22',
  'competition_id': 'sr:competition:33656'},
 {'id': 'sr:season:81062',
  'name': 'Premier League 2021',
  'start_date': '2020-12-12',
  'end_date': '2021-05-31',
  'year': '2021',
  'competition_id': 'sr:competition:33656'},
 {'id': 'sr:season:69034',
  'name': 'Copa Premier Centroamericana 2019',
  'start_date': '2019-07-10',
  'end_date': '2020-01-31',
  'year': '2019',
  'competition_id': 'sr:competition:29466'},
 {'id': 'sr:season:84150',
  'name': 'Canadian Premier League 2021',
  'start_date': '2021-06-26',
  'end_date': '2021-11-28',
  'year': '2021',
  'competition_id': 'sr:competition:28432'},
 {'id': 'sr:season:76379',
  'name': 'Canadian Premier League 2020',
  'start_date': '2020-04-11',
  'end_date': '2020-09-30',
  'year': '2020',
  'competition_id': 'sr:competition:28432'}]

I'll obtain the tournament results for the 2020--2021 season using the "sr:season:77179" competition ID.

In [3]:
competition_id = "sr:season:77179"

In [4]:
def get_season_summary(competition_id, soccer_key):
    address = "https://api.sportradar.us/soccer/trial/v4/en/seasons/" + competition_id + "/summaries.json?api_key=" + soccer_key
    return requests.get(address).json()

season_summary = get_season_summary(competition_id, soccer_key)
season_summary["summaries"][0]

{'sport_event': {'id': 'sr:sport_event:23203787',
  'start_time': '2020-09-12T11:30:00+00:00',
  'start_time_confirmed': True,
  'sport_event_context': {'sport': {'id': 'sr:sport:1', 'name': 'Soccer'},
   'category': {'id': 'sr:category:1',
    'name': 'England',
    'country_code': 'ENG'},
   'competition': {'id': 'sr:competition:17',
    'name': 'Premier League',
    'gender': 'men'},
   'season': {'id': 'sr:season:77179',
    'name': 'Premier League 20/21',
    'start_date': '2020-09-12',
    'end_date': '2021-05-24',
    'year': '20/21',
    'competition_id': 'sr:competition:17'},
   'stage': {'order': 1,
    'type': 'league',
    'phase': 'regular season',
    'start_date': '2020-09-12',
    'end_date': '2021-05-24',
    'year': '20/21'},
   'round': {'number': 1},
   'groups': [{'id': 'sr:league:51387', 'name': 'Premier League 20/21'}]},
  'coverage': {'type': 'sport_event',
   'sport_event_properties': {'lineups': True,
    'extended_player_stats': True,
    'extended_team_stats

To collect the information for the standings table, I store each match result into one dictionary.

Note: in soccer leagues, winning a match results in 3 points for the winner and 0 points for the loser. Drawing results in 1 point for each team. Goal difference is the difference between goals for and goals against.

In [6]:
from collections import defaultdict

def get_team_results(season_summary):
    team_results = defaultdict(list)
    for match in season_summary["summaries"]:
        
        # I collect the home/away team names for the match
        home_team, away_team = match['sport_event']['competitors'][0]['name'], match['sport_event']['competitors'][1]['name']
        
        # home_team = home_team_abbreviation?
        home_team, away_team = match['sport_event']['competitors'][0]['abbreviation'], match['sport_event']['competitors'][1]['abbreviation']

        # I skip over any matches which never occurred
        if match["sport_event_status"]['status'] == 'postponed':
            continue

        # collect the home and away scores
        home_score = match['sport_event_status']['home_score']
        away_score = match['sport_event_status']['away_score']
        # collect the home and away points
        if home_score > away_score:
            home_points = 3
            away_points = 0
        if away_score > home_score:
            away_points = 3
            home_points = 0
        if away_score == home_score:
            away_points = 1
            home_points = 1

        team_results[home_team].append({"scored" : home_score,
                                        "conceded" : away_score,
                                        "points" : home_points})
        team_results[away_team].append({"scored" : away_score,
                                        "conceded" : home_score,
                                        "points" : away_points})
    return team_results

team_results = get_team_results(season_summary)
team_results["ARS"][:5]

[{'scored': 3, 'conceded': 0, 'points': 3},
 {'scored': 2, 'conceded': 1, 'points': 3},
 {'scored': 1, 'conceded': 3, 'points': 0},
 {'scored': 2, 'conceded': 1, 'points': 3},
 {'scored': 0, 'conceded': 1, 'points': 0}]

Using all the matches in the accessd data, I create aggregate result data for each team and put it into a list. I collect the following data: the abbreviation of team `name`, the `games_played`, games `won`, games `drawn`, games `lost`, `goals_for`, `goals_against`, `goal_difference`, and total `points`.

In [7]:
def compute_standings(team_results):
    standings = []
    for team in team_results:                                       #team = ARS, FUL.....
        name = 0; won = 0; drawn = 0; lost = 0
        goals_for = 0; goals_against = 0; points = 0
        games_played = 0; goal_difference = 0

        numgames = []
        scoredgoals = []
        conc = []    
        # name
        name = team  
        
        # game is: {'scored': 3, 'conceded': 0, 'points': 3}
        for game in team_results[team]:                              
            
            games_played += 1
            #points and games_played
            numgames.append(game['points'])
            #games_played = len(numgames)
            points = sum(numgames)
            
#           #drawn
            
            if game['scored'] == game['conceded']:
                drawn += 1
                
#           #won and lost
            won= 0
            lost = 0
            for num in numgames:
                if num == 3:
                    won += 1
                if num == 0:
                    lost += 1
                    
#             #goals_for
            scoredgoals.append(game['scored'])
            goals_for = sum(scoredgoals)
            
#             #goals_against
            conc.append(game['conceded'])
            goals_against =sum(conc)
            
#             #goal_difference
            goal_difference = goals_for - goals_against

        standings.append([name, games_played, won, drawn, lost, 
                          goals_for, goals_against, goal_difference, points])
    return standings

standings = compute_standings(team_results)
standings

[['FUL', 18, 2, 6, 10, 15, 27, -12, 12],
 ['ARS', 20, 9, 3, 8, 26, 20, 6, 30],
 ['PAL', 20, 6, 5, 9, 24, 36, -12, 23],
 ['SOT', 19, 8, 5, 6, 27, 24, 3, 29],
 ['LFC', 19, 9, 7, 3, 37, 22, 15, 34],
 ['LEE', 19, 8, 2, 9, 32, 35, -3, 26],
 ['WHU', 20, 10, 5, 5, 30, 24, 6, 35],
 ['NEW', 20, 5, 4, 11, 19, 34, -15, 19],
 ['WBA', 20, 2, 5, 13, 15, 48, -33, 11],
 ['LEI', 19, 12, 2, 5, 35, 21, 14, 38],
 ['TOT', 18, 9, 6, 3, 33, 17, 16, 33],
 ['EVE', 17, 10, 2, 5, 28, 21, 7, 32],
 ['SHU', 19, 1, 2, 16, 10, 32, -22, 5],
 ['WOL', 20, 6, 5, 9, 21, 29, -8, 23],
 ['BHA', 19, 3, 8, 8, 22, 29, -7, 17],
 ['CFC', 20, 8, 6, 6, 33, 23, 10, 30],
 ['MNU', 19, 12, 4, 3, 36, 25, 11, 40],
 ['BUR', 19, 6, 4, 9, 13, 24, -11, 22],
 ['VIL', 18, 9, 2, 7, 33, 21, 12, 29],
 ['MNC', 19, 12, 5, 2, 36, 13, 23, 41]]

In the English Premier League, standings are calculated on the basis of points earned. I want to examine how points earned as a result of game-level performance compares to the overall number of goals earned.

First, I sort the standings list by the number of `goals_for` to see which team was the most effective at scoring in the tournament. 

In [202]:
standings = sorted(standings,reverse = True, key = lambda x: x[5])
standings

[['LFC', 19, 9, 7, 3, 37, 22, 15, 34],
 ['MNU', 19, 12, 4, 3, 36, 25, 11, 40],
 ['MNC', 19, 12, 5, 2, 36, 13, 23, 41],
 ['LEI', 19, 12, 2, 5, 35, 21, 14, 38],
 ['TOT', 18, 9, 6, 3, 33, 17, 16, 33],
 ['CFC', 20, 8, 6, 6, 33, 23, 10, 30],
 ['VIL', 18, 9, 2, 7, 33, 21, 12, 29],
 ['LEE', 19, 8, 2, 9, 32, 35, -3, 26],
 ['WHU', 20, 10, 5, 5, 30, 24, 6, 35],
 ['EVE', 17, 10, 2, 5, 28, 21, 7, 32],
 ['SOT', 19, 8, 5, 6, 27, 24, 3, 29],
 ['ARS', 20, 9, 3, 8, 26, 20, 6, 30],
 ['PAL', 20, 6, 5, 9, 24, 36, -12, 23],
 ['BHA', 19, 3, 8, 8, 22, 29, -7, 17],
 ['WOL', 20, 6, 5, 9, 21, 29, -8, 23],
 ['NEW', 20, 5, 4, 11, 19, 34, -15, 19],
 ['FUL', 18, 2, 6, 10, 15, 27, -12, 12],
 ['WBA', 20, 2, 5, 13, 15, 48, -33, 11],
 ['BUR', 19, 6, 4, 9, 13, 24, -11, 22],
 ['SHU', 19, 1, 2, 16, 10, 32, -22, 5]]

Next, I sort the standings list by the number of `points` to see who won the tournament. 

In [203]:
standings = sorted(standings,reverse = True, key = lambda x: x[8])
standings

[['MNC', 19, 12, 5, 2, 36, 13, 23, 41],
 ['MNU', 19, 12, 4, 3, 36, 25, 11, 40],
 ['LEI', 19, 12, 2, 5, 35, 21, 14, 38],
 ['WHU', 20, 10, 5, 5, 30, 24, 6, 35],
 ['LFC', 19, 9, 7, 3, 37, 22, 15, 34],
 ['TOT', 18, 9, 6, 3, 33, 17, 16, 33],
 ['EVE', 17, 10, 2, 5, 28, 21, 7, 32],
 ['CFC', 20, 8, 6, 6, 33, 23, 10, 30],
 ['ARS', 20, 9, 3, 8, 26, 20, 6, 30],
 ['VIL', 18, 9, 2, 7, 33, 21, 12, 29],
 ['SOT', 19, 8, 5, 6, 27, 24, 3, 29],
 ['LEE', 19, 8, 2, 9, 32, 35, -3, 26],
 ['PAL', 20, 6, 5, 9, 24, 36, -12, 23],
 ['WOL', 20, 6, 5, 9, 21, 29, -8, 23],
 ['BUR', 19, 6, 4, 9, 13, 24, -11, 22],
 ['NEW', 20, 5, 4, 11, 19, 34, -15, 19],
 ['BHA', 19, 3, 8, 8, 22, 29, -7, 17],
 ['FUL', 18, 2, 6, 10, 15, 27, -12, 12],
 ['WBA', 20, 2, 5, 13, 15, 48, -33, 11],
 ['SHU', 19, 1, 2, 16, 10, 32, -22, 5]]

The team that scored the most points was not the same team that won the tournament!

If two teams have the same points, the team with the higher goal difference is ranked higher. If the goal differences are also same, the team with the higher number of goals scored is ranked higher. To see the correct ranking from the tournament, I sort with consideration for all three factors, i.e., `points`, `goal_difference`, and `goals_for`.

In [206]:
standings = sorted(standings,reverse = True, key = lambda x: (x[8], x[7], x[5]))
standings

[['MNC', 19, 12, 5, 2, 36, 13, 23, 41],
 ['MNU', 19, 12, 4, 3, 36, 25, 11, 40],
 ['LEI', 19, 12, 2, 5, 35, 21, 14, 38],
 ['WHU', 20, 10, 5, 5, 30, 24, 6, 35],
 ['LFC', 19, 9, 7, 3, 37, 22, 15, 34],
 ['TOT', 18, 9, 6, 3, 33, 17, 16, 33],
 ['EVE', 17, 10, 2, 5, 28, 21, 7, 32],
 ['CFC', 20, 8, 6, 6, 33, 23, 10, 30],
 ['ARS', 20, 9, 3, 8, 26, 20, 6, 30],
 ['VIL', 18, 9, 2, 7, 33, 21, 12, 29],
 ['SOT', 19, 8, 5, 6, 27, 24, 3, 29],
 ['LEE', 19, 8, 2, 9, 32, 35, -3, 26],
 ['WOL', 20, 6, 5, 9, 21, 29, -8, 23],
 ['PAL', 20, 6, 5, 9, 24, 36, -12, 23],
 ['BUR', 19, 6, 4, 9, 13, 24, -11, 22],
 ['NEW', 20, 5, 4, 11, 19, 34, -15, 19],
 ['BHA', 19, 3, 8, 8, 22, 29, -7, 17],
 ['FUL', 18, 2, 6, 10, 15, 27, -12, 12],
 ['WBA', 20, 2, 5, 13, 15, 48, -33, 11],
 ['SHU', 19, 1, 2, 16, 10, 32, -22, 5]]

Below is the final standings table.
The fields are: abbreviation of team name, games played, games won, games drawn, games lost, goals for, goals against, goal difference, and total points. 

In [207]:
print("Team\tG\tW\tD\tL\tGF\tGA\tGD\tP")
for team in standings:
    line = "\t".join(map(str, team))
    print(line)

Team	G	W	D	L	GF	GA	GD	P
MNC	19	12	5	2	36	13	23	41
MNU	19	12	4	3	36	25	11	40
LEI	19	12	2	5	35	21	14	38
WHU	20	10	5	5	30	24	6	35
LFC	19	9	7	3	37	22	15	34
TOT	18	9	6	3	33	17	16	33
EVE	17	10	2	5	28	21	7	32
CFC	20	8	6	6	33	23	10	30
ARS	20	9	3	8	26	20	6	30
VIL	18	9	2	7	33	21	12	29
SOT	19	8	5	6	27	24	3	29
LEE	19	8	2	9	32	35	-3	26
WOL	20	6	5	9	21	29	-8	23
PAL	20	6	5	9	24	36	-12	23
BUR	19	6	4	9	13	24	-11	22
NEW	20	5	4	11	19	34	-15	19
BHA	19	3	8	8	22	29	-7	17
FUL	18	2	6	10	15	27	-12	12
WBA	20	2	5	13	15	48	-33	11
SHU	19	1	2	16	10	32	-22	5
